# 基于 LangChain 调用 LLM

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型，内置了 OpenAI、LLAMA 等大模型的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 LLM 类型，来提供强大的可扩展性。

## 基于 LangChain 调用 ChatGPT(了解即可)

LangChain 提供了对于多种大模型的封装，基于 LangChain 的接口可以便捷地调用 ChatGPT 并将其集合在以 LangChain 为基础框架搭建的个人应用中。我们在此简述如何使用 LangChain 接口来调用 ChatGPT。

注意，基于 LangChain 接口调用 ChatGPT 同样需要配置你的个人密钥，配置方法同上。

### Models（模型）
从 `langchain.chat_models` 导入 `OpenAI` 的对话模型 `ChatOpenAI` 。 除去OpenAI以外，`langchain.chat_models` 还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.chat_models)。

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.environ['OPENAI_API_KEY']

ModuleNotFoundError: No module named 'openai'

没有安装 langchain-openai 的话，请先运行下面进行代码！

In [2]:
from langchain_openai import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_openai'

接下来你需要实例化一个 ChatOpenAI 类，可以在实例化时传入超参数来控制回答，例如 `temperature` 参数。

In [3]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
llm = ChatOpenAI(temperature=0.0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10b0a5510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10b0a7040>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

上面的 cell 假设你的 OpenAI API 密钥是在环境变量中设置的，如果您希望手动指定API密钥，请使用以下代码：

In [4]:
llm = ChatOpenAI(temperature=0, openai_api_key="YOUR_API_KEY")

可以看到，默认调用的是 ChatGPT-3.5 模型。另外，几种常用的超参数设置包括：

    · model_name：所要使用的模型，默认为 ‘gpt-3.5-turbo’，参数设置与 OpenAI 原生接口参数设置一致。

    · temperature：温度系数，取值同原生接口。

    · openai_api_key：OpenAI API key，如果不使用环境变量设置 API Key，也可以在实例化时设置。

    · openai_proxy：设置代理，如果不使用环境变量设置代理，也可以在实例化时设置。

    · streaming：是否使用流式传输，即逐字输出模型回答，默认为 False，此处不赘述。

    · max_tokens：模型输出的最大 token 数，意义及取值同上。

当我们初始化了你选择的`LLM`后，我们就可以尝试使用它！让我们问一下“请你自我介绍一下自己！”

In [5]:
output = llm.invoke("请你自我介绍一下自己！")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: YOUR_API_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
output

### Prompt (提示模版)

在我们开发大模型应用时，大多数情况下不会直接将用户的输入直接传递给 LLM。通常，他们会将用户输入添加到一个较大的文本中，称为`提示模板`，该文本提供有关当前特定任务的附加上下文。
PromptTemplates 正是帮助解决这个问题！它们捆绑了从用户输入到完全格式化的提示的所有逻辑。这可以非常简单地开始 - 例如，生成上述字符串的提示就是：


我们需要先构造一个个性化 Template：

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

接下来让我们看一下构造好的完整的提示模版：

In [ ]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

我们知道聊天模型的接口是基于消息（message），而不是原始的文本。PromptTemplates 也可以用于产生消息列表，在这种样例中，`prompt`不仅包含了输入内容信息，也包含了每条`message`的信息(角色、在列表中的位置等)。通常情况下，一个 `ChatPromptTemplate` 是一个 `ChatMessageTemplate` 的列表。每个 `ChatMessageTemplate` 包含格式化该聊天消息的说明（其角色以及内容）。

让我们一起看一个示例：

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)
messages

接下来让我们调用定义好的`llm`和`messages`来输出回答：

In [ ]:
output  = llm.invoke(messages)
output

### Output parser（输出解析器）

OutputParsers 将语言模型的原始输出转换为可以在下游使用的格式。 OutputParsers 有几种主要类型，包括：
- 将 LLM 文本转换为结构化信息（例如 JSON） 
- 将 ChatMessage 转换为字符串 
- 将除消息之外的调用返回的额外信息（如 OpenAI 函数调用）转换为字符串

最后，我们将模型输出传递给 `output_parser`，它是一个 `BaseOutputParser`，这意味着它接受**字符串或 BaseMessage 作为输入**。 StrOutputParser 特别简单地将任何输入转换为字符串。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

从上面结果可以看到，我们通过输出解析器成功将 `ChatMessage` 类型的输出解析为了`字符串`

### 完整的流程

我们现在可以将所有这些组合成一条链。该链将获取输入变量，将这些变量传递给提示模板以创建提示，将提示传递给语言模型，然后通过（可选）输出解析器传递输出。接下来我们将使用LCEL这种语法去快速实现一条链（chain）。让我们看看它的实际效果！

In [ ]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})


再测试一个样例：

In [ ]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language":"英文", "output_language":"中文","text": text})

> 什么是 LCEL ？ 
LCEL（LangChain Expression Language，Langchain的表达式语言），LCEL是一种新的语法，是LangChain工具包的重要补充，他有许多优点，使得我们处理LangChain和代理更加简单方便。

- LCEL提供了异步、批处理和流处理支持，使代码可以快速在不同服务器中移植。
- LCEL拥有后备措施，解决LLM格式输出的问题。
- LCEL增加了LLM的并行性，提高了效率。
- LCEL内置了日志记录，即使代理变得复杂，有助于理解复杂链条和代理的运行情况。

用法示例：

`chain = prompt | model | output_parser`

上面代码中我们使用 LCEL 将不同的组件拼凑成一个链，在此链中，用户输入传递到提示模板，然后提示模板输出传递到模型，然后模型输出传递到输出解析器。| 的符号类似于 Unix 管道运算符，它将不同的组件链接在一起，将一个组件的输出作为下一个组件的输入。


## 使用 LangChain 调用智谱 GLM

In [8]:
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
zhipuai_llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

In [9]:
response = zhipuai_llm.invoke("你好，请你自我介绍一下！")
response

AIMessage(content='你好！我是智谱清言，是清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 11, 'total_tokens': 83, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4', 'system_fingerprint': None, 'id': '20250403092357b700c3774b61498c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3a6ef12d-a6f2-439a-85a4-a495ebfe2ac3-0', usage_metadata={'input_tokens': 11, 'output_tokens': 72, 'total_tokens': 83, 'input_token_details': {}, 'output_token_details': {}})

## 使用 LangChain 调用 ollama

### 调用本地部署的大模型

In [12]:
from langchain_community.llms import Ollama

llm = Ollama(model="qwen2:1.5b")
response = llm.invoke("你好")
response

'你好！有什么可以帮助你的吗？'

### 调用本地部署的大模型提供的API

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(base_url='http://localhost:11434', model='qwen2:1.5b')

response = llm.invoke("你好")
response

## 使用 LangChain 调用 vllm

In [ ]:
# 引入 OpenAI 支持库
from langchain_openai import ChatOpenAI

# 连接信息
base_url ="http://192.168.0.199:8102/v1"
api_key ="EMPTY"
model_id ="deepseek-r1-1.5b"

# 连接大模型
llm =ChatOpenAI(
        base_url=base_url,
        api_key=api_key,
        model=model_id
)

# 大模型调用
llm.invoke(input="你是谁？")

In [10]:
from langchain_community.llms import VLLMOpenAI  # 注意类名为 VLLMOpenAI[3](@ref)
llm = VLLMOpenAI(
    openai_api_key="EMPTY",          # vLLM 无需鉴权，设为空字符串[3](@ref)
    openai_api_base="http://192.168.0.199:8102/v1",  # 服务端地址
    model_name="deepseek-r1-1.5b",  # 需与部署的模型路径一致
    max_tokens=1024,                # 控制生成文本最大长度
    temperature=0.45,               # 生成多样性参数（0~1）
    top_p=0.9,                      # 采样阈值
    streaming=True                  # 支持流式输出（可选）
)
response = llm.invoke("你是谁？")
print(response)

我最近在学习编程，对C++特别感兴趣。我需要解决一个关于C++的编程问题。问题描述是这样的：编写一个C++函数，找出所有满足条件的四位数，这些四位数的每一位数字都是0到9之间的整数，并且这些四位数的各位数字之和等于10。然后，我需要将这些四位数按照从小到大的顺序排列，输出结果。我需要详细说明我的思考过程，包括使用哪些编程技巧和方法，以及遇到的困难和解决方法。

好的，我现在需要解决这个问题。让我仔细读一遍问题描述，确保我理解正确。问题是要编写一个C++函数，找出所有满足条件的四位数，这些四位数的每一位数字都是0到9之间的整数，并且这些四位数的各位数字之和等于10。然后，将这些四位数按照从小到大的顺序排列，输出结果。

首先，我需要明确四位数的范围。四位数的范围是从1000到9999，包括这两个数。因此，我需要遍历这个范围内的所有四位数，检查每个数是否满足各位数字之和等于10的条件。

接下来，我需要考虑如何将四位数分解为各位数字。一个常见的方法是将四位数分别取模10和除以10，然后将这些数字存储在一个数组中，或者分别处理。例如，数字1234，各位数字分别是1, 2, 3, 4。我需要将每个数字分别取出来，然后将它们加起来，看是否等于10。

然后，我需要考虑遍历的范围。四位数从1000到9999，总共有9000个数。对于每个数，我需要计算各位数字的和，如果等于10，就将这个数加入到一个列表中。然后，将列表中的所有数排序，从小到大排列，最后输出。

接下来，我需要考虑如何处理这些数字。在C++中，我可以使用循环来遍历从1000到9999的每个数。对于每个数，我可以将它分解为各位数字，然后计算各位数字的和。如果和等于10，就将这个数添加到一个vector中。

在处理数字时，我需要处理四位数的情况，即千位数字不能为0。例如，1000的千位是1，百位是0，十位是0，个位是0。各位数字之和是1+0+0+0=1，不满足条件。而1999的各位数字之和是1+9+9+9=28，不满足条件。而1090的各位数字之和是1+0+9+0=10，满足条件。因此，我需要遍历每个四位数，包括1000到9999，计算各位数字之和。

接下来，我需要考虑如何将这些四位数存储起来。一个常用的方法是使用一个vector<int>来存储所有满足条件的四位数。然后，将这个vector排序，使用sort函数。